In [4]:
# 找40万平衡数据

import pandas as pd

df = pd.read_csv("E:/作业/杂物/web日志分析/log01.csv", skiprows=range(1, 3000001),  nrows=3000000)
# 随机抽样（可改 n=1000000 抽少一点）
df_sample = df.sample(n=3000000, random_state=36)

# 保存抽样结果
df_sample.to_csv("E:/作业/杂物/web日志分析/sampled_004.csv", index=False)


In [5]:
import pandas as pd

df = pd.read_csv("E:/作业/杂物/web日志分析/sampled_004.csv")  # 读取数据

df['score'] = 0
df['label'] = 0  # 默认都是正常的，后续高分的我们再打成 1

for i, row in df.iterrows():  #和上边的评分规则一模一样
    score = 0
    path = str(row['path']) if pd.notna(row['path']) else ""
    status = str(row['status']) if pd.notna(row['status']) else ""
    user_agent = str(row['user_agent']) if pd.notna(row['user_agent']) else ""

    if len(path) > 100:
        score += 2
    dangerous_words = ['script', 'union', 'select', 'drop', 'admin', '..', 'passwd']
    for word in dangerous_words:
        if word.lower() in path.lower():
            score += 3
    if status.startswith('4') or status.startswith('5'):
        score += 1
    if len(user_agent) < 20:
        score += 2
    special_chars = ['<', '>', '%', '&', ';']
    for char in special_chars:
        if char in path:
            score += 1

    df.at[i, 'score'] = score
    df.at[i, 'label'] = 1 if score >= 3 else 0  # 可疑打为1，正常为0


df.to_csv("E:/作业/杂物/web日志分析/sampled_004_labeled.csv", index=False, encoding='utf-8-sig')  # 保存为新的带伪标签的数据集

print("打标签完成")

打标签完成


In [6]:
df_labeled = pd.read_csv("E:/作业/杂物/web日志分析/sampled_004_labeled.csv")

df_bad = df_labeled[df_labeled['label'] == 1].sample(n=200000, random_state=42)
df_good = df_labeled[df_labeled['label'] == 0].sample(n=200000, random_state=42)

df_final = pd.concat([df_bad, df_good]).sample(frac=1, random_state=42)  # 打乱顺序
df_final.to_csv("E:/作业/杂物/web日志分析/final_dataset111.csv", index=False)
